Implement a user‐based nearest neighbor recommendation algorithm. Write a program that:  
A) Accepts a user ID as an input (on the console),  
B) Then shows the titles and genres of up to 15 movies that this user has rated
,

In [1]:
import pandas as pd
import numpy as np

user_input = int(input("Insert user ID"))    #remember to convert the input to int!!

df_ratings = pd.read_table("ratings.dat", sep="::", header=None, names=["user_id","movie_id","rating","timestamp"], encoding="ISO-8859-1")

df_movies = pd.read_table("movies.dat", sep="::", names=["movie_id","title", "genres"], encoding="ISO-8859-1")

if user_input not in df_ratings["user_id"].values:
    print(f"User {user_input} not found in the ratings dataset")
else:
    merged_df = df_ratings.merge(df_movies, how="left")

    print(f"User {user_input} has rated:")

    print(merged_df[merged_df.user_id == user_input][["title", "genres"]].head(15))


C:\Users\gianl\AppData\Local\Temp\ipykernel_13776\4147592488.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_ratings = pd.read_table("ratings.dat", sep="::", header=None, names=["user_id","movie_id","rating","timestamp"], encoding="ISO-8859-1")
C:\Users\gianl\AppData\Local\Temp\ipykernel_13776\4147592488.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_movies = pd.read_table("movies.dat", sep="::", names=["movie_id","title", "genres"], encoding="ISO-8859-1")


User 3 has rated:
                                                 title  \
182                                Animal House (1978)   
183                             Full Monty, The (1997)   
184                         Mission: Impossible (1996)   
185                             Raising Arizona (1987)   
186                                     28 Days (2000)   
187                               Happy Gilmore (1996)   
188                           Golden Child, The (1986)   
189  Star Wars: Episode VI - Return of the Jedi (1983)   
190                         Beverly Hills Ninja (1997)   
191  Naked Gun: From the Files of Police Squad!, Th...   
192                        Fish Called Wanda, A (1988)   
193                        Being John Malkovich (1999)   
194                                   Edge, The (1997)   
195          Indiana Jones and the Last Crusade (1989)   
196                                 Stand by Me (1986)   

                                  genres  
182       

Implement a user‐based nearest neighbor recommendation algorithm. Write a program that:  
A) Accepts a user ID as an input (on the console),  
B) Then shows the titles and genres of up to 15 movies that this user has rated

C) Then displays the 10 movies with the highest predicted relevance score according to the nearest neighbor technique

In [2]:
# df_users = pd.read_table("users.dat", sep="::", names=["user_id","gender", "age", "occupation", "zip_code"], encoding="ISO-8859-1")

# merged_df2 = merged_df.merge(df_users, how="left")

user_item_table = pd.pivot_table(merged_df, values="rating", index="user_id", columns="title", fill_value=np.nan)

def cosine_similarity(A, B):
    # Find common elements
    common_elements = np.intersect1d(np.nonzero(~np.isnan(A)), np.nonzero(~np.isnan(B)))
    if len(common_elements) == 0:
        return 0.0
    # Compute cosine similarity between common elements
    A_common = A[common_elements]
    B_common = B[common_elements]
    cosine_sim = np.dot(A_common, B_common) / (np.linalg.norm(A_common) * np.linalg.norm(B_common))
    return cosine_sim


def get_similar_users (user, user_item_table, n):
    similar_users = []
    for index, row in user_item_table.iterrows():
        if index == user:
            continue    # pass to next user if it's the same as the current one
        similarity = cosine_similarity (user_item_table.loc[user].values, user_item_table.loc[index].values)
        similar_users.append((index, similarity))       #Remember the double parentheses!
    similar_users = sorted(similar_users, key=lambda x: x[1], reverse=True)
    
    return similar_users[:n]
    

def get_recommendations(user, user_item_table, k=10):
    similar_users = get_similar_users(user, user_item_table, k)

    predicted_ratings = {}
    for title in user_item_table.columns :
        if pd.isnull(user_item_table.loc[user,title]): #checking if the input user has already rated the current film
            predicted_rating = 0
            total_similarity = 0
            for similar_user, similarity in similar_users:
                if not pd.isnull(user_item_table.loc[similar_user,title]): #checking if the similar user has already rated the current film
                    predicted_rating += similarity * user_item_table.loc[similar_user, title]
                    total_similarity += similarity

            if total_similarity > 0:
                predicted_rating /= total_similarity
            predicted_ratings[title] = predicted_rating

    recommendations = sorted(predicted_ratings.items(), key=lambda x:x[1], reverse=True )
    return recommendations[:k]

print(get_recommendations(user_input, user_item_table))


[('African Queen, The (1951)', 5.0), ('All About My Mother (Todo Sobre Mi Madre) (1999)', 5.0), ('All Over Me (1997)', 5.0), ('American History X (1998)', 5.0), ('American Movie (1999)', 5.0), ('Amityville Horror, The (1979)', 5.0), ('Baby, The (1973)', 5.0), ('Best Man, The (1999)', 5.0), ("Boys Don't Cry (1999)", 5.0), ('Cell, The (2000)', 5.0)]
